In [1]:
import zipfile
from google.colab import drive

In [2]:
import os
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!ls drive/My\ Drive/Datasets

CatsDogs.zip


In [5]:
folders = os.listdir("/content/drive/My Drive/Datasets")
print(folders)

['CatsDogs.zip']


In [6]:
!unzip /content/drive/My\ Drive/Datasets/CatsDogs.zip -d CatDog

Archive:  /content/drive/My Drive/Datasets/CatsDogs.zip
  inflating: CatDog/dataset/cats/cat.1.jpg  
  inflating: CatDog/dataset/cats/cat.10.jpg  
  inflating: CatDog/dataset/cats/cat.100.jpg  
  inflating: CatDog/dataset/cats/cat.101.jpg  
  inflating: CatDog/dataset/cats/cat.102.jpg  
  inflating: CatDog/dataset/cats/cat.103.jpg  
  inflating: CatDog/dataset/cats/cat.104.jpg  
  inflating: CatDog/dataset/cats/cat.105.jpg  
  inflating: CatDog/dataset/cats/cat.106.jpg  
  inflating: CatDog/dataset/cats/cat.107.jpg  
  inflating: CatDog/dataset/cats/cat.108.jpg  
  inflating: CatDog/dataset/cats/cat.109.jpg  
  inflating: CatDog/dataset/cats/cat.11.jpg  
  inflating: CatDog/dataset/cats/cat.110.jpg  
  inflating: CatDog/dataset/cats/cat.111.jpg  
  inflating: CatDog/dataset/cats/cat.112.jpg  
  inflating: CatDog/dataset/cats/cat.113.jpg  
  inflating: CatDog/dataset/cats/cat.114.jpg  
  inflating: CatDog/dataset/cats/cat.115.jpg  
  inflating: CatDog/dataset/cats/cat.116.jpg  
  inflat

In [9]:
!rm -rf CatDog/dataset/dataset

In [11]:
folders = os.listdir("/content/CatDog/dataset")
print(folders)

['horses', 'dogs', 'cats', 'humans']


In [14]:
 image_data = []
 labels = []

 label_dict = {
     "cats":0,
     "dogs":1,
     "horses":2,
     "humans":3
 }

In [19]:
from keras.preprocessing import image

In [20]:
for ix in folders:
  path = os.path.join("/content/CatDog/dataset",ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im),target_size=((224,224)))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])
  print(path)

/content/CatDog/dataset/horses
/content/CatDog/dataset/dogs
/content/CatDog/dataset/cats
/content/CatDog/dataset/humans


In [21]:
print(len(image_data),len(labels))

808 808


In [23]:
import random
combined = list(zip(image_data,labels))
random.shuffle(combined)

image_data[:],labels[:] = zip(*combined)

In [24]:
print(labels[:5])

[2, 0, 3, 0, 1]


In [26]:
import numpy as np
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808,)


In [27]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [28]:
 #Create the Model Resnet 50 Model

 from keras.applications.resnet50 import ResNet50
 from keras.optimizers import Adam
 from keras.layers import *
 from keras.models import Model

 import matplotlib.pyplot as plt

In [32]:
model = ResNet50(include_top = False,weights='imagenet',input_shape=(224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step


In [33]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [34]:
av1 = GlobalAveragePooling2D()(model.output)

fc1 = Dense(256,activation='relu')(av1)

d1 = Dropout(0.5)(fc1)

fc2 = Dense(4,activation='softmax')(d1)


model_new = Model(input = model.input ,output= fc2)

model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [35]:
adam = Adam(lr=0.00003)
model_new.compile(loss ='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [38]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f1a0c2113c8>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1a0c100be0>
2 <keras.layers.convolutional.Conv2D object at 0x7f1a0c100b00>
3 <keras.layers.normalization.BatchNormalization object at 0x7f1a0c100a58>
4 <keras.layers.core.Activation object at 0x7f1a0c1007f0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1a650c65c0>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f1a0c19fac8>
7 <keras.layers.convolutional.Conv2D object at 0x7f1a0c0767f0>
8 <keras.layers.normalization.BatchNormalization object at 0x7f1a0039fe48>
9 <keras.layers.core.Activation object at 0x7f1a0035ca90>
10 <keras.layers.convolutional.Conv2D object at 0x7f1a0035c940>
11 <keras.layers.normalization.BatchNormalization object at 0x7f1a0036f358>
12 <keras.layers.core.Activation object at 0x7f1a00382c88>
13 <keras.layers.convolutional.Conv2D object at 0x7f1a00316e80>
14 <keras.layers.convolutional.Conv2D object at 0x7f1a0032d358>
15 <keras.lay

In [37]:
for ix in range(169):
  model_new.layers[ix].trainable = False

In [39]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f1a0c2113c8>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1a0c100be0>
2 <keras.layers.convolutional.Conv2D object at 0x7f1a0c100b00>
3 <keras.layers.normalization.BatchNormalization object at 0x7f1a0c100a58>
4 <keras.layers.core.Activation object at 0x7f1a0c1007f0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1a650c65c0>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f1a0c19fac8>
7 <keras.layers.convolutional.Conv2D object at 0x7f1a0c0767f0>
8 <keras.layers.normalization.BatchNormalization object at 0x7f1a0039fe48>
9 <keras.layers.core.Activation object at 0x7f1a0035ca90>
10 <keras.layers.convolutional.Conv2D object at 0x7f1a0035c940>
11 <keras.layers.normalization.BatchNormalization object at 0x7f1a0036f358>
12 <keras.layers.core.Activation object at 0x7f1a00382c88>
13 <keras.layers.convolutional.Conv2D object at 0x7f1a00316e80>
14 <keras.layers.convolutional.Conv2D object at 0x7f1a0032d358>
15 <keras.lay

In [40]:
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [41]:
hist = model_new.fit(X_train,Y_train,
                     shuffle = True,
                     batch_size = 16,
                     epochs=5,
                     validation_split = 0.20
                     )


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 39s 60ms/step - loss: 0.9949 - accuracy: 0.5991 - val_loss: 0.3011 - val_accuracy: 0.8827
Epoch 2/5
646/646 [==============================] - 15s 24ms/step - loss: 0.2212 - accuracy: 0.9427 - val_loss: 0.1675 - val_accuracy: 0.9321
Epoch 3/5
646/646 [==============================] - 15s 24ms/step - loss: 0.0850 - accuracy: 0.9892 - val_loss: 0.1155 - val_accuracy: 0.9506
Epoch 4/5
646/646 [==============================] - 15s 24ms/step - loss: 0.0583 - accuracy: 0.9907 - val_loss: 0.1096 - val_accuracy: 0.9568
Epoch 5/5
646/646 [==============================] - 15s 23ms/step - loss: 0.0368 - accuracy: 0.9969 - val_loss: 0.1132 - val_accuracy: 0.9568
